In [7]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 


In [15]:
# Load input data and labels

dataset = datasets.load_breast_cancer()

x, y = dataset.data, dataset.target

print(x.shape, y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

(569, 30) (569,)


In [ ]:
def evaluated(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    print(f"Precision: {precision_score(y_true, y_pred)}")
    print(f"Recall: {recall_score(y_true, y_pred)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print(f"Confusion Matrix: \n{confusion_matrix(y_true, y_pred)}")

In [ ]:
# Implement Logistic Regression using libraries

from sklearn.linear_model import LogisticRegression

regressor = LogisticRegression(max_iter=2000)
regressor.fit(X_train, Y_train)

# Predict
predictions = regressor.predict(X_test)

# Evaluate

accuracy = np.sum(predictions == Y_test) / Y_test.shape[0]



In [ ]:
# Implement Logistic Regressor class from scratch

class Logistic_regressor(n_Samples=2000, learning_rate=0.01):

    def _init_(self, n_Samples=2000, learning_rate=0.01):
        self.n_samples = n_Samples
        self.learnig_rate = learning_rate
        self.weights = None
        self.bias = None
        self.losses = []